In [1]:
import os
import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, make_scorer, recall_score
from PIL import Image
from glob import glob
from sklearn.svm import SVC

In [ ]:
def load_images_and_masks_gray(folder, img_ext='jpg', mask_ext='png'):
    X = []
    y = []
    img_paths = sorted(glob(os.path.join(folder, f'*.{img_ext}')))
    for img_path in img_paths:
        base = os.path.splitext(os.path.basename(img_path))[0]
        
        if base.endswith('_BCI'):
            mask_base = base[:-4]  
        else:
            mask_base = base
        
        mask_path = os.path.join(folder, f'{mask_base}.{mask_ext}')
        if os.path.exists(mask_path):
            img = np.array(Image.open(img_path).convert('L'))   
            mask = np.array(Image.open(mask_path).convert('L'))
            mask_bin = (mask > 127).astype(np.uint8)
            X.append(img.reshape(-1, 1))                     
            y.append(mask_bin.flatten())
        else:
            print(f'No mask found for {img_path}')
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    return X, y

In [ ]:
def load_images_and_masks(folder, img_ext='jpg', mask_ext='png'):
    X = []
    y = []
    img_paths = sorted(glob(os.path.join(folder, f'*.{img_ext}')))
    for img_path in img_paths:
        base = os.path.splitext(os.path.basename(img_path))[0]
        
        if base.endswith('_BCI'):
            mask_base = base[:-4]
        else:
            mask_base = base
        
        mask_path = os.path.join(folder, f'{mask_base}.{mask_ext}')
        if os.path.exists(mask_path):
            img = np.array(Image.open(img_path).convert('RGB'))
            mask = np.array(Image.open(mask_path).convert('L'))
            mask_bin = (mask > 127).astype(np.uint8)
            X.append(img.reshape(-1, 3))
            y.append(mask_bin.flatten())
        else:
            print(f'No mask found for {img_path}')
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    return X, y



In [ ]:
train_imgs = 'train_dir'

X, y = load_images_and_masks(train_imgs)

In [ ]:
param_grid = {'reg_param': np.linspace(0, 1, 11)}

qda = QuadraticDiscriminantAnalysis()

grid = GridSearchCV(qda, param_grid, cv=3, n_jobs=-1, scoring='balanced_accuracy')
grid.fit(X, y)

In [ ]:

y_pred = grid.predict(X)
print(classification_report(y, y_pred))

In [ ]:
y_pred = grid.predict(X)
print(classification_report(y, y_pred))

In [ ]:
test_folder = 'test_dir'
test_imgs = [f for f in os.listdir(test_folder) if f.endswith('.jpg')]

modelo = grid.best_estimator_ 

for test_img in test_imgs:
    img_path = os.path.join(test_folder, test_img)
    img = Image.open(img_path).convert('RGB')
    img_np = np.array(img)
    h, w, c = img_np.shape
    
    X_test = img_np.reshape(-1, 3)
    y_pred = modelo.predict(X_test)
    
    mask_pred = y_pred.reshape(h, w).astype(np.uint8) * 255
    
    pred_mask_img = Image.fromarray(mask_pred)
    pred_mask_img.save(os.path.join(test_folder, f'{os.path.splitext(test_img)[0]}_qda_mask.png'))


## Segmentation BCI

In [ ]:
train_imgs = 'train_bci_dir'

X, y = load_images_and_masks_gray(train_imgs)

In [ ]:
param_grid = {
    'reg_param': np.linspace(0, 1, 11) 
}

qda = QuadraticDiscriminantAnalysis()
grid = GridSearchCV(qda, param_grid, cv=3, scoring = 'roc_auc')
grid.fit(X, y)

In [ ]:
y_pred = grid.predict(X)
print(classification_report(y, y_pred))

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
test_folder = 'test_BCI_image'
test_imgs = [f for f in os.listdir(test_folder) if f.endswith('.jpg')]

modelo = grid.best_estimator_ 

for test_img in test_imgs:
    img_path = os.path.join(test_folder, test_img)
    img = Image.open(img_path).convert('L') 
    img_np = np.array(img)
    h, w = img_np.shape 
    
    X_test = img_np.reshape(-1, 1) 
    y_pred = modelo.predict(X_test)
    
    mask_pred = y_pred.reshape(h, w).astype(np.uint8) * 255

    pred_mask_img = Image.fromarray(mask_pred)
    pred_mask_img.save(os.path.join(test_folder, f'{os.path.splitext(test_img)[0]}_qda_mask.png'))
